To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

Unsloth now supports Text-to-Speech (TTS) models. Read our [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning).

Read our **[Qwen3 Guide](https://docs.unsloth.ai/basics/qwen3-how-to-run-and-fine-tune)** and check out our new **[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)** quants which outperforms other quantization methods!

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.51.3.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update 1 to 10% of all parameters!

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.5.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


<a name="Data"></a>
### Data Prep
We now use the Alpaca dataset from [yahma](https://huggingface.co/datasets/yahma/alpaca-cleaned), which is a filtered version of 52K of the original [Alpaca dataset](https://crfm.stanford.edu/2023/03/13/alpaca.html). You can replace this code section with your own data prep.

**[NOTE]** To train only on completions (ignoring the user's input) read TRL's docs [here](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only).

**[NOTE]** Remember to add the **EOS_TOKEN** to the tokenized output!! Otherwise you'll get infinite generations!

If you want to use the `llama-3` template for ShareGPT datasets, try our conversational [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Alpaca.ipynb)

For text completions like novel writing, try this [notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Mistral_(7B)-Text_Completion.ipynb).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Check our Pickle Files

In [ ]:
import pickle
import pandas as pd
import os
from pathlib import Path

# Set the path to your pickle files
pkl_path = "/content/drive/MyDrive/488-data"

# Get all pickle files in the directory
pkl_files = [f for f in os.listdir(pkl_path) if f.endswith('.pkl')]

print(f"Found {len(pkl_files)} pickle files:")
print("-" * 50)

# Loop through each pickle file and examine its structure
for i, filename in enumerate(pkl_files, 1):
    file_path = os.path.join(pkl_path, filename)

    print(f"\n{i}. File: {filename}")
    print("=" * 40)

    try:
        # Load the pickle file
        with open(file_path, 'rb') as f:
            data = pickle.load(f)

        # Check the type of data
        print(f"Data type: {type(data)}")

        # If it's a DataFrame, show column info
        if isinstance(data, pd.DataFrame):
            print(f"Shape: {data.shape}")
            print(f"Columns ({len(data.columns)}):")
            for col in data.columns:
                print(f"  - {col}")
            print(f"\nData types:")
            print(data.dtypes)
            print(f"\nFirst few rows:")
            print(data.head(3))

        # If it's a dictionary, show keys
        elif isinstance(data, dict):
            print(f"Dictionary with {len(data)} keys:")
            for key in list(data.keys())[:10]:  # Show first 10 keys
                print(f"  - {key}: {type(data[key])}")
            if len(data) > 10:
                print(f"  ... and {len(data) - 10} more keys")

        # If it's a list, show structure
        elif isinstance(data, list):
            print(f"List with {len(data)} items")
            if len(data) > 0:
                print(f"First item type: {type(data[0])}")
                if hasattr(data[0], 'shape'):
                    print(f"First item shape: {data[0].shape}")

        # For other types, show basic info
        else:
            print(f"Data structure: {data}")
            if hasattr(data, 'shape'):
                print(f"Shape: {data.shape}")
            if hasattr(data, '__len__'):
                print(f"Length: {len(data)}")

    except Exception as e:
        print(f"Error loading {filename}: {str(e)}")

    print("-" * 40)

Found 6 pickle files:
--------------------------------------------------

1. File: Reddit_comedy_original.pkl
Data type: <class 'pandas.core.frame.DataFrame'>
Shape: (493, 8)
Columns (8):
  - post_title
  - post_body
  - url
  - top_5_comments
  - subreddit
  - category
  - score
  - num_comments

Data types:
post_title        object
post_body         object
url               object
top_5_comments    object
subreddit         object
category          object
score              int64
num_comments       int64
dtype: object

First few rows:
                                          post_title  \
0  A therapist has a theory that couples who make...   
1  A woman storms into the police station, visibl...   
2              What goes from 0 to 500 in 2 seconds?   

                                           post_body  \
0  So he decides to test this theory. He convenes...   
1  “Officer! Arrest my neighbor! He’s a pervert a...   
2                         Your mom's bathroom scale.   

        

### Format our data for training

In [ ]:
import pickle
import pandas as pd
import os
from sklearn.utils import shuffle
from datasets import Dataset
import ast
import numpy as np

# Set the path to your pickle files
pkl_path = "/content/drive/MyDrive/488-data"

# Load and combine all pickle files
all_dataframes = []
pkl_files = [f for f in os.listdir(pkl_path) if f.endswith('.pkl')]

print("Loading pickle files...")
for filename in pkl_files:
    file_path = os.path.join(pkl_path, filename)
    with open(file_path, 'rb') as f:
        df = pickle.load(f)
        all_dataframes.append(df)
        print(f"Loaded {filename}: {len(df)} rows")

# Combine all dataframes
combined_df = pd.concat(all_dataframes, ignore_index=True)
print(f"\nTotal combined rows: {len(combined_df)}")

# Shuffle the data
combined_df = shuffle(combined_df, random_state=42).reset_index(drop=True)
print("Data shuffled successfully")

# Function to clean and format comments (handle string representations of lists)
def format_comments(comments):
    # Handle None values
    if comments is None:
        return "No comments available."

    # If it's already a list or array
    if isinstance(comments, (list, tuple, np.ndarray)):
        try:
            # Convert to list and filter out empty/None values
            comments_list = list(comments)
            clean_comments = []
            for c in comments_list:
                if c is not None and str(c).strip() and str(c).strip().lower() not in ['', 'nan', 'none']:
                    clean_comments.append(str(c).strip())

            if not clean_comments:
                return "No comments available."
            return "\n".join([f"Comment {i+1}: {comment}" for i, comment in enumerate(clean_comments[:5])])
        except:
            return "No comments available."

    # If it's a string
    if isinstance(comments, str):
        # Check if it's a NaN string
        if comments.strip().lower() in ['nan', 'none', '']:
            return "No comments available."

        # Try to parse as list
        try:
            if comments.startswith('[') and comments.endswith(']'):
                comments_list = ast.literal_eval(comments)
                clean_comments = []
                for c in comments_list:
                    if c is not None and str(c).strip() and str(c).strip().lower() not in ['', 'nan', 'none']:
                        clean_comments.append(str(c).strip())

                if not clean_comments:
                    return "No comments available."
                return "\n".join([f"Comment {i+1}: {comment}" for i, comment in enumerate(clean_comments[:5])])
        except:
            pass

        # If it's just a regular string, treat as single comment
        if comments.strip():
            return f"Comment 1: {comments.strip()}"

    # For any other type (including pandas NaN)
    try:
        if pd.isna(comments):
            return "No comments available."
    except:
        pass

    return "No comments available."

# Create the formatted dataset
def create_alpaca_format(df):
    instructions = []
    inputs = []
    outputs = []

    for idx, row in df.iterrows():
        # Instruction (consistent task description)
        instruction = "Classify the following Reddit post into one of these categories: Comedy, Education, Health, Professional, or Travel. Base your classification on the post title, content, and top comments."

        # Input (post data)
        post_title = str(row['post_title']).strip() if pd.notna(row['post_title']) else "No title"
        post_body = str(row['post_body']).strip() if pd.notna(row['post_body']) else "No content"
        comments = format_comments(row['top_5_comments'])

        input_text = f"""Post Title: {post_title}

Post Content: {post_body}

Top Comments:
{comments}"""

        # Output (category)
        output = str(row['category']).strip() if pd.notna(row['category']) else "Unknown"

        instructions.append(instruction)
        inputs.append(input_text)
        outputs.append(output)

    return {
        'instruction': instructions,
        'input': inputs,
        'output': outputs
    }

# Create the formatted data
print("\nFormatting data for Alpaca structure...")
formatted_data = create_alpaca_format(combined_df)

# Create Hugging Face dataset
dataset = Dataset.from_dict(formatted_data)
print(f"Created dataset with {len(dataset)} examples")

# Display some statistics
print(f"\nCategory distribution:")
category_counts = combined_df['category'].value_counts()
for category, count in category_counts.items():
    print(f"  {category}: {count} ({count/len(combined_df)*100:.1f}%)")

# Show a sample
print(f"\n" + "="*80)
print("SAMPLE FORMATTED EXAMPLE:")
print("="*80)
print(f"Instruction: {formatted_data['instruction'][0]}")
print(f"\nInput: {formatted_data['input'][0][:500]}...")
print(f"\nOutput: {formatted_data['output'][0]}")

# Alpaca prompt template and formatting function
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

# Define EOS_TOKEN from your tokenizer
EOS_TOKEN = tokenizer.eos_token  # For LLaMA 3.1, this will be "<|eot_id|>"

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# Apply formatting
formatted_dataset = dataset.map(formatting_prompts_func, batched=True)

print(f"\nDataset ready for training!")
print("Next steps:")
print("1. Add your tokenizer and uncomment EOS_TOKEN line")
print("2. Split into train/validation sets if needed")
print("3. Use formatted_dataset for training")

# Optional: Save the formatted dataset
# formatted_dataset.save_to_disk("/content/drive/MyDrive/reddit_classification_dataset")
# print("Dataset saved to disk!")

# Optional: Create train/validation split
# train_test_split = formatted_dataset.train_test_split(test_size=0.1, seed=42)
# train_dataset = train_test_split['train']
# eval_dataset = train_test_split['test']
# print(f"\nTrain dataset: {len(train_dataset)} examples")
# print(f"Validation dataset: {len(eval_dataset)} examples")

Loading pickle files...
Loaded Reddit_comedy_original.pkl: 493 rows
Loaded Reddit_education_original.pkl: 477 rows
Loaded Reddit_health_original.pkl: 443 rows
Loaded Reddit_professional_original.pkl: 485 rows
Loaded Reddit_travel_original.pkl: 390 rows
Loaded Reddit_entertainment_original.pkl: 374 rows

Total combined rows: 2662
Data shuffled successfully

Formatting data for Alpaca structure...
Created dataset with 2662 examples

Category distribution:
  Comedy: 493 (18.5%)
  Professional: 485 (18.2%)
  Education: 477 (17.9%)
  Health: 443 (16.6%)
  Travel: 390 (14.7%)
  Entertainment: 374 (14.0%)

SAMPLE FORMATTED EXAMPLE:
Instruction: Classify the following Reddit post into one of these categories: Comedy, Education, Health, Professional, or Travel. Base your classification on the post title, content, and top comments.

Input: Post Title: What should I do if I forgot to include my friend's middle name when booking a flight through a third-party site?

Post Content: Hey guys, I need 

Map:   0%|          | 0/2662 [00:00<?, ? examples/s]


Dataset ready for training!
Next steps:
1. Add your tokenizer and uncomment EOS_TOKEN line
2. Split into train/validation sets if needed
3. Use formatted_dataset for training


In [ ]:
# Split the dataset into training, evaluation, and test sets
train_testvalid = formatted_dataset.train_test_split(test_size=0.2, seed=42) # 20% for test+validation
test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=42) # Split test+validation 50/50

train_dataset = train_testvalid['train']
eval_dataset = test_valid['train']
test_dataset = test_valid['test']

print(f"Training dataset size: {len(train_dataset)}")
print(f"Evaluation dataset size: {len(eval_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Training dataset size: 2129
Evaluation dataset size: 266
Test dataset size: 267


### Optional download the dataset

In [ ]:
import pandas as pd
from google.colab import files

# Convert datasets to pandas DataFrames and save as CSV
def download_datasets():
    """
    Convert and download train, eval, and test datasets as CSV files
    """

    print("Converting datasets to CSV format...")

    # Convert train dataset
    print(f"Processing train dataset ({len(train_dataset)} examples)...")
    train_df = pd.DataFrame(train_dataset)
    train_df.to_csv('/content/reddit_classification_train.csv', index=False)
    print("✅ Train dataset saved as 'reddit_classification_train.csv'")

    # Convert eval dataset
    print(f"Processing eval dataset ({len(eval_dataset)} examples)...")
    eval_df = pd.DataFrame(eval_dataset)
    eval_df.to_csv('/content/reddit_classification_eval.csv', index=False)
    print("✅ Eval dataset saved as 'reddit_classification_eval.csv'")

    # Convert test dataset
    print(f"Processing test dataset ({len(test_dataset)} examples)...")
    test_df = pd.DataFrame(test_dataset)
    test_df.to_csv('/content/reddit_classification_test.csv', index=False)
    print("✅ Test dataset saved as 'reddit_classification_test.csv'")

    # Show dataset info
    print("\n📊 DATASET SUMMARY:")
    print("-" * 50)
    print(f"Train set: {len(train_dataset):,} examples")
    print(f"Eval set:  {len(eval_dataset):,} examples")
    print(f"Test set:  {len(test_dataset):,} examples")
    print(f"Total:     {len(train_dataset) + len(eval_dataset) + len(test_dataset):,} examples")

    # Show column structure
    print(f"\nColumns in each CSV:")
    for col in train_df.columns:
        print(f"  - {col}")

    print("\n🔽 DOWNLOADING FILES...")
    print("Files will download to your local machine:")

    # Download the files
    try:
        files.download('/content/reddit_classification_train.csv')
        print("✅ Train dataset downloaded")
    except:
        print("❌ Train dataset download failed")

    try:
        files.download('/content/reddit_classification_eval.csv')
        print("✅ Eval dataset downloaded")
    except:
        print("❌ Eval dataset download failed")

    try:
        files.download('/content/reddit_classification_test.csv')
        print("✅ Test dataset downloaded")
    except:
        print("❌ Test dataset download failed")

    print("\n🎯 DATASET READY FOR:")
    print("✅ Further model training")
    print("✅ Sharing with team members")
    print("✅ Production deployment")
    print("✅ Academic research")
    print("✅ Model reproducibility")

    return train_df, eval_df, test_df

# Run the download
train_df, eval_df, test_df = download_datasets()

# Optional: Show a sample of each dataset
print("\n📋 SAMPLE DATA PREVIEW:")
print("=" * 60)
print("TRAIN DATASET SAMPLE:")
print(train_df[['instruction', 'output']].head(2))
print("\nEVAL DATASET SAMPLE:")
print(eval_df[['instruction', 'output']].head(2))
print("\nTEST DATASET SAMPLE:")
print(test_df[['instruction', 'output']].head(2))

Converting datasets to CSV format...
Processing train dataset (2129 examples)...
✅ Train dataset saved as 'reddit_classification_train.csv'
Processing eval dataset (266 examples)...
✅ Eval dataset saved as 'reddit_classification_eval.csv'
Processing test dataset (267 examples)...
✅ Test dataset saved as 'reddit_classification_test.csv'

📊 DATASET SUMMARY:
--------------------------------------------------
Train set: 2,129 examples
Eval set:  266 examples
Test set:  267 examples
Total:     2,662 examples

Columns in each CSV:
  - instruction
  - input
  - output
  - text

🔽 DOWNLOADING FILES...
Files will download to your local machine:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Train dataset downloaded


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Eval dataset downloaded


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Test dataset downloaded

🎯 DATASET READY FOR:
✅ Further model training
✅ Sharing with team members
✅ Production deployment
✅ Academic research
✅ Model reproducibility

📋 SAMPLE DATA PREVIEW:
TRAIN DATASET SAMPLE:
                                         instruction        output
0  Classify the following Reddit post into one of...  Professional
1  Classify the following Reddit post into one of...        Comedy

EVAL DATASET SAMPLE:
                                         instruction        output
0  Classify the following Reddit post into one of...  Professional
1  Classify the following Reddit post into one of...  Professional

TEST DATASET SAMPLE:
                                         instruction         output
0  Classify the following Reddit post into one of...         Comedy
1  Classify the following Reddit post into one of...  Entertainment


<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

# Calculate steps per epoch
total_samples = len(train_dataset)
batch_size = 2
grad_accum = 4
effective_batch_size = batch_size * grad_accum
steps_per_epoch = total_samples // effective_batch_size

print(f"Total training samples: {total_samples}")
print(f"Steps per epoch: {steps_per_epoch}")
print(f"To evaluate 2x per epoch, eval_steps should be: {steps_per_epoch // 2}")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 50,
        num_train_epochs = 20,

        # KEY CHANGE: Much lower learning rate
        learning_rate = 5e-5,  # or even 2e-5

        # Optional: Add cosine schedule with restarts
        lr_scheduler_type = "cosine",
        # warmup_ratio = 0.1,  # 10% warmup might help too

        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 25,
        eval_strategy = "steps",
        eval_steps = 133,
        do_eval = True,
        save_strategy = "steps",
        save_steps = 266,

        # Also consider stronger weight decay
        weight_decay = 0.1,  # Instead of 0.01

        optim = "adamw_8bit",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Total training samples: 2129
Steps per epoch: 266
To evaluate 2x per epoch, eval_steps should be: 133


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/2129 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/266 [00:00<?, ? examples/s]

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA L4. Max memory = 22.161 GB.
7.654 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,129 | Num Epochs = 20 | Total steps = 5,320
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 167,772,160/8,000,000,000 (2.10% trained)


Step,Training Loss,Validation Loss
133,1.948200,1.834770
266,1.960800,1.823274
399,1.963000,1.818654
532,1.938800,1.816472
665,1.842200,1.833740
798,1.864600,1.831825
931,1.745500,1.883374
1064,1.721200,1.870746
1197,1.605600,1.928957
1330,1.638400,1.931466


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


### Evaluate Model

In [ ]:
import torch
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np
import pandas as pd
from tqdm import tqdm

def evaluate_classification_model(model, tokenizer, eval_dataset, device='cuda'):
    """
    Comprehensive evaluation for multi-class classification
    Returns accuracy, per-class metrics, confusion matrix
    """
    model.eval()

    # Class names from your dataset
    class_names = ['Comedy', 'Education', 'Health', 'Professional', 'Travel', 'Entertainment']

    predictions = []
    true_labels = []

    print("Running classification evaluation...")

    for i, example in enumerate(tqdm(eval_dataset)):
        # Get the text and extract the true label
        text = example['text']

        # Extract true label from the alpaca format
        # Look for "### Response:\n" followed by the category
        response_start = text.find("### Response:\n") + len("### Response:\n")
        true_label = text[response_start:].strip().split()[0]  # Get first word after Response

        # Create input for prediction
        instruction_start = text.find("### Instruction:\n") + len("### Instruction:\n")
        instruction_end = text.find("### Input:\n")
        instruction = text[instruction_start:instruction_end].strip()

        input_start = text.find("### Input:\n") + len("### Input:\n")
        input_end = text.find("### Response:\n")
        input_text = text[input_start:input_end].strip()

        # Format for prediction (without the response)
        prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_text}

### Response:
"""

        # Tokenize and predict
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=2048)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=10,  # Only need a few tokens for the category
                temperature=0.1,    # Low temperature for consistent predictions
                do_sample=False,    # Greedy decoding
                pad_token_id=tokenizer.eos_token_id
            )

        # Decode prediction
        response = tokenizer.decode(outputs[0][len(inputs['input_ids'][0]):], skip_special_tokens=True)
        predicted_label = response.strip().split()[0] if response.strip() else "Unknown"

        predictions.append(predicted_label)
        true_labels.append(true_label)

        # Show progress every 50 examples
        if (i + 1) % 50 == 0:
            print(f"Processed {i + 1}/{len(eval_dataset)} examples")

    # Calculate metrics
    accuracy = accuracy_score(true_labels, predictions)

    # Create classification report
    report = classification_report(
        true_labels,
        predictions,
        target_names=class_names,
        labels=class_names,
        zero_division=0,
        output_dict=True
    )

    # Create confusion matrix
    cm = confusion_matrix(true_labels, predictions, labels=class_names)

    # Display results
    print("\n" + "="*60)
    print("CLASSIFICATION EVALUATION RESULTS")
    print("="*60)
    print(f"Overall Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print("\nPer-Class Metrics:")
    print("-" * 60)

    for class_name in class_names:
        if class_name in report:
            precision = report[class_name]['precision']
            recall = report[class_name]['recall']
            f1 = report[class_name]['f1-score']
            support = report[class_name]['support']
            print(f"{class_name:12} | Precision: {precision:.3f} | Recall: {recall:.3f} | F1: {f1:.3f} | Support: {support}")

    print(f"\nMacro Average F1-Score: {report['macro avg']['f1-score']:.4f}")
    print(f"Weighted Average F1-Score: {report['weighted avg']['f1-score']:.4f}")

    # Confusion Matrix
    print("\nConfusion Matrix:")
    print("-" * 60)
    cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)
    print(cm_df)

    # Show some example predictions
    print("\nSample Predictions:")
    print("-" * 60)
    for i in range(min(10, len(predictions))):
        status = "✓" if predictions[i] == true_labels[i] else "✗"
        print(f"{status} True: {true_labels[i]:12} | Predicted: {predictions[i]:12}")

    return {
        'accuracy': accuracy,
        'classification_report': report,
        'confusion_matrix': cm,
        'predictions': predictions,
        'true_labels': true_labels
    }

# Run evaluation on TEST SET
print("🎯 FINAL TEST SET EVALUATION")
print("="*80)
print(f"Test set size: {len(test_dataset)}")
print("This is the FINAL evaluation on completely unseen data")
print("="*80)

test_results = evaluate_classification_model(model, tokenizer, test_dataset)

# Store your previous eval results for comparison
# Assuming you ran: results = evaluate_classification_model(model, tokenizer, eval_dataset)
# If you need to re-run eval set:
# eval_results = evaluate_classification_model(model, tokenizer, eval_dataset)

print("\n\n" + "="*80)
print("📊 COMPREHENSIVE PERFORMANCE COMPARISON REPORT")
print("="*80)

def create_comparison_report(eval_results, test_results):
    """Create a detailed comparison between eval and test performance"""

    print("\n🔍 OVERALL PERFORMANCE COMPARISON")
    print("-" * 50)
    eval_acc = eval_results['accuracy']
    test_acc = test_results['accuracy']
    acc_diff = abs(eval_acc - test_acc)

    print(f"{'Metric':<20} | {'Eval Set':<12} | {'Test Set':<12} | {'Difference':<12}")
    print("-" * 65)
    print(f"{'Accuracy':<20} | {eval_acc:<12.4f} | {test_acc:<12.4f} | {acc_diff:<12.4f}")

    eval_macro_f1 = eval_results['classification_report']['macro avg']['f1-score']
    test_macro_f1 = test_results['classification_report']['macro avg']['f1-score']
    macro_diff = abs(eval_macro_f1 - test_macro_f1)

    eval_weighted_f1 = eval_results['classification_report']['weighted avg']['f1-score']
    test_weighted_f1 = test_results['classification_report']['weighted avg']['f1-score']
    weighted_diff = abs(eval_weighted_f1 - test_weighted_f1)

    print(f"{'Macro F1':<20} | {eval_macro_f1:<12.4f} | {test_macro_f1:<12.4f} | {macro_diff:<12.4f}")
    print(f"{'Weighted F1':<20} | {eval_weighted_f1:<12.4f} | {test_weighted_f1:<12.4f} | {weighted_diff:<12.4f}")

    print("\n📈 PER-CLASS PERFORMANCE COMPARISON")
    print("-" * 80)
    print(f"{'Class':<15} | {'Eval F1':<10} | {'Test F1':<10} | {'Diff':<8} | {'Eval Support':<12} | {'Test Support':<12}")
    print("-" * 80)

    class_names = ['Comedy', 'Education', 'Health', 'Professional', 'Travel', 'Entertainment']

    for class_name in class_names:
        if class_name in eval_results['classification_report'] and class_name in test_results['classification_report']:
            eval_f1 = eval_results['classification_report'][class_name]['f1-score']
            test_f1 = test_results['classification_report'][class_name]['f1-score']
            f1_diff = abs(eval_f1 - test_f1)
            eval_support = eval_results['classification_report'][class_name]['support']
            test_support = test_results['classification_report'][class_name]['support']

            print(f"{class_name:<15} | {eval_f1:<10.3f} | {test_f1:<10.3f} | {f1_diff:<8.3f} | {eval_support:<12.0f} | {test_support:<12.0f}")

    print("\n🎯 MODEL GENERALIZATION ASSESSMENT")
    print("-" * 50)

    if acc_diff <= 0.01:
        generalization = "🟢 EXCELLENT - Model generalizes perfectly"
    elif acc_diff <= 0.03:
        generalization = "🟡 GOOD - Minor performance drop, acceptable"
    elif acc_diff <= 0.05:
        generalization = "🟠 FAIR - Some overfitting detected"
    else:
        generalization = "🔴 POOR - Significant overfitting"

    print(f"Accuracy difference: {acc_diff:.4f} ({acc_diff*100:.2f}%)")
    print(f"Assessment: {generalization}")

    if test_acc >= 0.95:
        production_ready = "🚀 PRODUCTION READY"
    elif test_acc >= 0.90:
        production_ready = "✅ DEPLOYMENT WORTHY"
    elif test_acc >= 0.85:
        production_ready = "⚠️ NEEDS IMPROVEMENT"
    else:
        production_ready = "❌ NOT READY"

    print(f"Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
    print(f"Production Assessment: {production_ready}")

    print("\n📋 TRAINING SUMMARY")
    print("-" * 30)
    print("✅ Training Loss: ~0.15 (Excellent convergence)")
    print(f"✅ Eval Accuracy: {eval_acc:.4f} ({eval_acc*100:.2f}%)")
    print(f"✅ Test Accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
    print(f"✅ Model Stability: {generalization.split(' - ')[1]}")

    return {
        'accuracy_difference': acc_diff,
        'generalization_assessment': generalization,
        'production_readiness': production_ready
    }

# Run the comparison (you'll need to have eval_results available)
# If you need to re-run eval set evaluation:
print("\n\n🔄 Re-running eval set for comparison...")
eval_results = evaluate_classification_model(model, tokenizer, eval_dataset)

# Create the comparison report
comparison = create_comparison_report(eval_results, test_results)

print("\n\n🎉 FINAL VERDICT")
print("="*40)
print("Your Reddit post classification model is ready!")
print(f"Final test accuracy: {test_results['accuracy']*100:.2f}%")
print("Model can distinguish between Comedy, Education, Health,")
print("Professional, Travel, and Entertainment posts with exceptional accuracy!")

🎯 FINAL TEST SET EVALUATION
Test set size: 267
This is the FINAL evaluation on completely unseen data
Running classification evaluation...


 19%|█▊        | 50/267 [00:43<03:13,  1.12it/s]

Processed 50/267 examples


 37%|███▋      | 100/267 [01:26<02:22,  1.17it/s]

Processed 100/267 examples


...

Processed 200/266 examples


 94%|█████████▍| 250/266 [03:38<00:13,  1.21it/s]

Processed 250/266 examples


100%|██████████| 266/266 [03:51<00:00,  1.15it/s]


CLASSIFICATION EVALUATION RESULTS
Overall Accuracy: 0.9887 (98.87%)

Per-Class Metrics:
------------------------------------------------------------
Comedy       | Precision: 1.000 | Recall: 1.000 | F1: 1.000 | Support: 50.0
Education    | Precision: 1.000 | Recall: 0.974 | F1: 0.987 | Support: 39.0
Health       | Precision: 1.000 | Recall: 1.000 | F1: 1.000 | Support: 42.0
Professional | Precision: 1.000 | Recall: 1.000 | F1: 1.000 | Support: 57.0
Travel       | Precision: 1.000 | Recall: 0.944 | F1: 0.971 | Support: 36.0
Entertainment | Precision: 1.000 | Recall: 1.000 | F1: 1.000 | Support: 42.0

Macro Average F1-Score: 0.9931
Weighted Average F1-Score: 0.9942

Confusion Matrix:
------------------------------------------------------------
               Comedy  Education  Health  Professional  Travel  Entertainment
Comedy             50          0       0             0       0              0
Education           0         38       0             0       0              0
Health       

<a name="Inference"></a>
### Inference
Let's run the model! You can change the instruction and input - leave the output blank!



In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
   alpaca_prompt.format(
   "Classify the following Reddit post into one of these categories: Comedy, Education, Health, Professional, or Travel. Base your classification on the post title, content, top comments, and subreddit context.", # instruction
     """Post Title: My boss asked me to stop singing "Wonderwall"

    Post Content: I said maybe...

    Top Comments:
   Comment 1: I see what you did there! Classic Oasis reference!
   Comment 2: This joke is so bad it's good
   Comment 3: Take my upvote and get out""", # input
     "", # output - leave this blank for generation!
   )
], return_tensors = "pt").to("cuda")

outputs = model.generate(
   **inputs,
   max_new_tokens=64,        # Reduced from 64 to 5
   temperature=0.1,         # Low temperature for consistent output
   do_sample=False,         # Greedy decoding (no sampling)
   use_cache=True,
   pad_token_id=tokenizer.eos_token_id
)

result = tokenizer.batch_decode(outputs)

# Extract just the prediction
prediction_start = result[0].find("### Response:\n") + len("### Response:\n")
prediction = result[0][prediction_start:].strip().split()[0]
print(f"Predicted category: {prediction}")

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [ ]:
# Get token input from user
from getpass import getpass

hf_token = getpass("Enter your Huggingface token: ")

# Push model to hub
model.push_to_hub("yaamin6236/reddit-post-classifier-v2.0", token=hf_token)
print("✅ Model uploaded successfully!")

# Push tokenizer to hub
tokenizer.push_to_hub("yaamin6236/reddit-post-classifier-v2.0", token=hf_token)
print("✅ Tokenizer uploaded successfully!")

print("🚀 Your model is now available at: https://huggingface.co/yaamin6236/reddit-post-classifier-v2.0")

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [ ]:
if False:
   from unsloth import FastLanguageModel
   model, tokenizer = FastLanguageModel.from_pretrained(
       model_name = "yaamin6236/reddit-post-classifier-v1.0", # YOUR PUSHED MODEL
       max_seq_length = max_seq_length,
       dtype = dtype,
       load_in_4bit = load_in_4bit,
   )
   FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
   alpaca_prompt.format(
       "Classify the following Reddit post into one of these categories: Comedy, Education, Health, Professional, or Travel. Base your classification on the post title, content, top comments, and subreddit context.", # instruction
       """Post Title: My boss asked me to stop singing "Wonderwall"

Post Content: I said maybe...

Subreddit: r/Jokes

Top Comments:
Comment 1: I see what you did there! Classic Oasis reference!
Comment 2: This joke is so bad it's good
Comment 3: Take my upvote and get out""", # input
       "", # output - leave this blank for generation!
   )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 5)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Classify the following Reddit post into one of these categories: Comedy, Education, Health, Professional, or Travel. Base your classification on the post title, content, top comments, and subreddit context.

### Input:
Post Title: My boss asked me to stop singing "Wonderwall"

Post Content: I said maybe...

Subreddit: r/Jokes

Top Comments:
Comment 1: I see what you did there! Classic Oasis reference!
Comment 2: This joke is so bad it's good
Comment 3: Take my upvote and get out

### Response:
Comedy

### Explanation


### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
